In [1]:
%matplotlib inline
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from plots import *
from kls import *
from nats import *
from utils import *
from objectives import *
from torch.distributions.normal import Normal
from torch.distributions.one_hot_categorical import OneHotCategorical as cat
from torch.distributions.gamma import Gamma
import sys
import time
import datetime
sys.path.append('/home/hao/Research/probtorch/')
import probtorch
from probtorch.util import expand_inputs
print('probtorch:', probtorch.__version__, 
      'torch:', torch.__version__, 
      'cuda:', torch.cuda.is_available())

probtorch: 0.0+5a2c637 torch: 1.0.0 cuda: True


In [2]:
N = 60
K = 3
D = 2

## Model Parameters
MCMC_SIZE = 10
SAMPLE_SIZE = 10
NUM_HIDDEN1 = 8

NUM_STATS = 1 + 2 * D
NUM_LATENTS =  D
## Training Parameters
SAMPLE_DIM = 0
BATCH_DIM = 1
BATCH_SIZE = 20
NUM_EPOCHS = 10000
LEARNING_RATE = 1e-3
CUDA = torch.cuda.is_available()
PATH = 'gibbs-z-encoder2'

In [3]:
Xs = torch.from_numpy(np.load('gmm_dataset/obs.npy')).float()
# STATES = torch.from_numpy(np.load('gmm_dataset/states.npy')).float()
OBS_MU = torch.from_numpy(np.load('gmm_dataset/obs_mu.npy')).float()
OBS_SIGMA = torch.from_numpy(np.load('gmm_dataset/obs_sigma.npy')).float()
Pi = torch.from_numpy(np.load('gmm_dataset/init.npy')).float()
NUM_SEQS = Xs.shape[0]
NUM_BATCHES = int((Xs.shape[0] / BATCH_SIZE))

In [4]:
class Enc_z(nn.Module):
    def __init__(self, num_obs=3*D,
                       num_hidden=NUM_HIDDEN1,
                       num_latents=K):
        super(self.__class__, self).__init__()
        self.pi_prob = nn.Sequential(
            nn.Linear(num_obs, num_hidden),
            nn.Tanh(),
            nn.Linear(num_hidden, int(0.5*num_hidden)),
            nn.Linear(int(0.5*num_hidden), 1))
        
        self.prior_pi = torch.ones(K) * (1./ K)
        if CUDA:
            self.prior_pi = self.prior_pi.cuda()
  
    def forward(self, obs, obs_tau, obs_mu, sample_size, batch_size):
        q = probtorch.Trace()
        obs_tau_c1 = obs_tau[:, :, 0, :].unsqueeze(-2).repeat(1,1,N,1)
        obs_mu_c1 = obs_mu[:, :, 0, :].unsqueeze(-2).repeat(1,1,N,1)
        obs_tau_c2 = obs_tau[:, :, 1, :].unsqueeze(-2).repeat(1,1,N,1)
        obs_mu_c2 = obs_mu[:, :, 1, :].unsqueeze(-2).repeat(1,1,N,1)
        obs_tau_c3 = obs_tau[:, :, 2, :].unsqueeze(-2).repeat(1,1,N,1)
        obs_mu_c3 = obs_mu[:, :, 2, :].unsqueeze(-2).repeat(1,1,N,1)
        
        data_c1 = torch.cat((obs, obs_mu_c1, obs_tau_c1), -1) ## S * B * N * 3D
        data_c2 = torch.cat((obs, obs_mu_c2, obs_tau_c2), -1) ## S * B * N * 3D
        data_c3 = torch.cat((obs, obs_mu_c3, obs_tau_c3), -1) ## S * B * N * 3D
        
        z_pi_c1 = self.pi_prob(data_c1)
        z_pi_c2 = self.pi_prob(data_c2)
        z_pi_c3 = self.pi_prob(data_c3)
        
        z_pi = F.softmax(torch.cat((z_pi_c1, z_pi_c2, z_pi_c3), -1), -1)
        z = cat(z_pi).sample()
        _ = q.variable(cat, probs=z_pi, value=z, name='zs')
        p = probtorch.Trace()
        _ = p.variable(cat, probs=self.prior_pi, value=z, name='zs')
        return q, p
    
def initialize():
    enc_z = Enc_z()
    if CUDA:
        enc_z.cuda()
    optimizer =  torch.optim.Adam(list(enc_z.parameters()),lr=LEARNING_RATE, betas=(0.9, 0.99))    
    return enc_z, optimizer

In [5]:
enc_z, optimizer = initialize()

In [6]:
def Eubo(enc_z, obs, obs_mu, obs_sigma, sample_size, batch_size):
    """
    gibbs sampler for z
    """
    ## update z -- cluster assignments
    q_z, p_z = enc_z(obs, obs_sigma, obs_mu, sample_size, batch_size)
    log_q_z = q_z['zs'].log_prob ## S * B * N
    log_p_z = p_z['zs'].log_prob ## S * B * N
    states = q_z['zs'].value ## S * B * N * K
    labels = states.argmax(-1)
    log_obs = Log_likelihood(obs, states, obs_mu, obs_sigma, K, D, cluster_flag=False)
    log_weights = log_obs + log_p_z - log_q_z # S * B * N
    weights = F.softmax(log_weights, 0).detach()
    eubo = (weights * log_weights).sum(0).sum(-1).mean()
    elbo = log_weights.sum(-1).mean()
    ess = (1. / (weights**2).sum(0)).mean(-1).mean()

    q_logits = q_z['zs'].dist.probs.log()
    ## KLs for cluster assignments
    post_logits = Post_z(obs, obs_sigma, obs_mu, N, K)
    kl_z_ex, kl_z_in = kls_cats(q_logits, post_logits)

    return eubo, elbo, ess, kl_z_ex.sum(-1).mean(), kl_z_in.sum(-1).mean()


In [7]:
EUBOs = []
ELBOs = []
ESSs = []
flog = open('results/log-' + PATH + '.txt', 'w+')
flog.write('EUBO\tELBO\tESS\t\KL_ex\tKL_in\n')
flog.close()
for epoch in range(NUM_EPOCHS):
    time_start = time.time()
    indices = torch.randperm(NUM_SEQS)
    EUBO = 0.0
    ELBO = 0.0
    ESS = 0.0
    KL_ex = 0.0
    KL_in = 0.0
    for step in range(NUM_BATCHES):
        optimizer.zero_grad()
        batch_indices = indices[step*BATCH_SIZE : (step+1)*BATCH_SIZE]
        obs = Xs[batch_indices]
        obs = shuffler(obs).repeat(SAMPLE_SIZE, 1, 1, 1)
        obs_mu = OBS_MU[batch_indices].repeat(SAMPLE_SIZE, 1, 1, 1)
        obs_sigma = OBS_SIGMA[batch_indices].repeat(SAMPLE_SIZE, 1, 1, 1)
        if CUDA:
            obs =obs.cuda()
            obs_mu = obs_mu.cuda()
            obs_sigma = obs_sigma.cuda()
        eubo, elbo, ess, kl_ex, kl_in = Eubo(enc_z, obs, obs_mu, obs_sigma, SAMPLE_SIZE, BATCH_SIZE)
        ## gradient step
        eubo.backward()
    
        optimizer.step()
        EUBO += eubo.item()
        ELBO += elbo.item()
        ESS += ess.item()
        KL_ex += kl_ex.item() 
        KL_in += kl_in.item()
        
    flog = open('results/log-' + PATH + '.txt', 'a+')
    print('%.3f\t%.3f\t%.3f\t%.3f\t%.3f'
            % (EUBO/NUM_BATCHES, ELBO/NUM_BATCHES, ESS/NUM_BATCHES, KL_ex/NUM_BATCHES, KL_in/NUM_BATCHES), file=flog)
    flog.close()
 
    time_end = time.time()
    print('epoch=%d, EUBO=%.3f, ELBO=%.3f, ESS=%.3f, z_ex=%.3f, z_in=%.3f (%ds)'
        % (epoch, EUBO/NUM_BATCHES, ELBO/NUM_BATCHES, ESS/NUM_BATCHES, 
           KL_ex/NUM_BATCHES, KL_in/NUM_BATCHES, time_end - time_start))

epoch=0, EUBO=-204.241, ELBO=-1747.202, ESS=4.712, z_ex=941.640, z_in=46.970 (2s)
epoch=1, EUBO=-219.962, ELBO=-792.113, ESS=7.344, z_ex=365.900, z_in=19.333 (1s)
epoch=2, EUBO=-233.496, ELBO=-333.689, ESS=9.188, z_ex=66.020, z_in=5.024 (1s)
epoch=3, EUBO=-236.948, ELBO=-252.205, ESS=9.735, z_ex=10.025, z_in=1.446 (1s)
epoch=4, EUBO=-237.540, ELBO=-242.670, ESS=9.844, z_ex=3.296, z_in=0.827 (2s)
epoch=5, EUBO=-237.808, ELBO=-240.314, ESS=9.891, z_ex=1.678, z_in=0.612 (2s)
epoch=6, EUBO=-237.893, ELBO=-239.395, ESS=9.913, z_ex=1.075, z_in=0.502 (1s)
epoch=7, EUBO=-237.951, ELBO=-239.081, ESS=9.927, z_ex=0.777, z_in=0.422 (2s)
epoch=8, EUBO=-238.003, ELBO=-238.871, ESS=9.939, z_ex=0.606, z_in=0.359 (2s)
epoch=9, EUBO=-238.034, ELBO=-238.719, ESS=9.947, z_ex=0.470, z_in=0.307 (2s)
epoch=10, EUBO=-238.055, ELBO=-238.601, ESS=9.953, z_ex=0.373, z_in=0.268 (2s)
epoch=11, EUBO=-238.063, ELBO=-238.514, ESS=9.957, z_ex=0.300, z_in=0.238 (1s)
epoch=12, EUBO=-238.074, ELBO=-238.470, ESS=9.960, z_

KeyboardInterrupt: 

In [ ]:
torch.save(enc_eta.state_dict(), 'weights/enc-%s' + PATH)

In [ ]:
def plot_results(EUBOs, ELBOs, ESSs, num_samples, num_epochs, lr):
    fig = plt.figure(figsize=(15, 15))
    fig.tight_layout()
    ax1 = fig.add_subplot(3, 1, 1)
    ax2 = fig.add_subplot(3, 1, 2)
    ax3 = fig.add_subplot(3, 1, 3)
    ax1.plot(EUBOs, 'r', label='EUBOs')
    ax1.plot(ELBOs, 'b', label='ELBOs')

    ax1.tick_params(labelsize=18)
    
    ax3.plot(np.array(ESSs) / num_samples, 'm', label='ESS')
    ax1.set_title('epoch=%d, batch_size=%d, lr=%.1E, samples=%d' % (num_epochs, BATCH_SIZE, lr, num_samples), fontsize=18)
    ax1.set_ylim([-240, -100])
    ax1.legend()
    ax3.legend()
    ax3.tick_params(labelsize=18)
    ax3.set_ylim([0,1])
    plt.savefig('train_' + PATH + '.svg')

In [ ]:
def sample_single_batch(num_seqs, N, K, D, batch_size):
    indices = torch.randperm(num_seqs)
    batch_indices = indices[0*batch_size : (0+1)*batch_size]
    obs = Xs[batch_indices]
    obs = shuffler(obs).repeat(SAMPLE_SIZE, 1, 1, 1)
    obs_mu = OBS_MU[batch_indices].repeat(SAMPLE_SIZE, 1, 1, 1)
    obs_sigma = OBS_SIGMA[batch_indices].repeat(SAMPLE_SIZE, 1, 1, 1)
    if CUDA:
        obs =obs.cuda()
        obs_mu = obs_mu.cuda()
        obs_sigma = obs_sigma.cuda()
    return obs, obs_mu, obs_sigma

def test(obs, obs_mu, obs_sigma, sample_size, batch_size):
    q_z, p_z = enc_z(obs, obs_sigma, obs_mu, sample_size, batch_size)
    return q_z
        
def plot_samples(obs, obs_mu, obs_sigma, q_z, batch_size):
    colors = ['r', 'b', 'gold']
    fig = plt.figure(figsize=(15,15))
    xs = obs[0].cpu()
    zs_mean = q_z['zs'].dist.probs.cpu()[0]
    mus = obs_mu[0].cpu().data.numpy()
    sigmas = obs_sigma[0].cpu().data.numpy()
    for b in range(batch_size):
        ax = fig.add_subplot(int(batch_size / 5), 5, b+1)
        x = xs[b].data.numpy()
        z = zs_mean[b].data.numpy()
        mu = mus[b]
        sigma = sigmas[b]
        assignments = z.argmax(-1)
        for k in range(K):
            cov_k = np.diag(sigma[k]**2)
            xk = x[np.where(assignments == k)]
            ax.scatter(xk[:, 0], xk[:, 1], c=colors[k])
            plot_cov_ellipse(cov=cov_k, pos=mu[k], nstd=2, ax=ax, alpha=0.2, color=colors[k])
        ax.set_ylim([-15, 15])
        ax.set_xlim([-15, 15])
    plt.savefig('results/modes-' + PATH + '.svg')        

In [ ]:
BATCH_SIZE_TEST = 25
obs, obs_mu, obs_sigma = sample_single_batch(NUM_SEQS, N, K, D, BATCH_SIZE_TEST)

In [ ]:
q_z = test(obs, obs_mu, obs_sigma, SAMPLE_SIZE, BATCH_SIZE_TEST)
plot_samples(obs, obs_mu, obs_sigma, q_z, BATCH_SIZE_TEST)
%time